In [1]:
import os
import numpy as np
import re
import keras
import tensorflow as tf

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


A function from set 6 to parse the observations:

In [2]:
def parse_observations(text):
    # Convert text to dataset.
    lines = [line.split() for line in text.split('\n') if line.split()]

    obs_counter = 0
    obs = []
    obs_map = {}

    for line in lines:
        obs_elem = []
        
        for word in line:
            word = re.sub(r'[^\w]', '', word).lower()
            if word not in obs_map:
                # Add unique words to the observations map.
                obs_map[word] = obs_counter
                obs_counter += 1
            
            # Add the encoded word.
            obs_elem.append(obs_map[word])
        
        # Add the encoded sequence.
        obs.append(obs_elem)

    return obs, obs_map

In [3]:
text = open(os.path.join(os.getcwd(), \
                         'Release/data/shakespeare.txt')).read()
obs, obs_map = parse_observations(text)

In [4]:
all_chars = list(text)
chars = sorted(list(set(text)))
data_size, vocab_size = len(text), len(chars)
print ('data has %d chars, %d unique' ,(data_size, vocab_size))

data has %d chars, %d unique (98029, 71)


In [5]:
char_to_index = {char:idx for idx, char in enumerate(chars)}

In [8]:
index_to_char = {idx:char for idx, char in enumerate(chars)}


In [61]:
print(index_to_char)

{0: '\n', 1: ' ', 2: '!', 3: "'", 4: '(', 5: ')', 6: ',', 7: '-', 8: '.', 9: '0', 10: '1', 11: '2', 12: '3', 13: '4', 14: '5', 15: '6', 16: '7', 17: '8', 18: '9', 19: ':', 20: ';', 21: '?', 22: 'A', 23: 'B', 24: 'C', 25: 'D', 26: 'E', 27: 'F', 28: 'G', 29: 'H', 30: 'I', 31: 'J', 32: 'K', 33: 'L', 34: 'M', 35: 'N', 36: 'O', 37: 'P', 38: 'R', 39: 'S', 40: 'T', 41: 'U', 42: 'V', 43: 'W', 44: 'Y', 45: 'a', 46: 'b', 47: 'c', 48: 'd', 49: 'e', 50: 'f', 51: 'g', 52: 'h', 53: 'i', 54: 'j', 55: 'k', 56: 'l', 57: 'm', 58: 'n', 59: 'o', 60: 'p', 61: 'q', 62: 'r', 63: 's', 64: 't', 65: 'u', 66: 'v', 67: 'w', 68: 'x', 69: 'y', 70: 'z'}


In [97]:
n_step = 2
s_len = 40
training_data = []
next_char = []
for i in range(0, len(text)-s_len, n_step):
    string_to_add = text[i:i + s_len]
    training_data.append(string_to_add)
    if i + s_len < len(text)-1:
        next_char.append(text[i + s_len])
    
# This is our end symbol $
next_char.append(' ')

In [98]:
print(len(training_data))
print(len(next_char))

48995
48995


In [99]:
trainX = np.zeros((len(training_data), s_len, len(chars)))
trainY = np.zeros((len(training_data), len(chars)))

for sent_idx in range(0, len(training_data)):
    curr_sentence = training_data[sent_idx]
    curr_next_char = next_char[sent_idx]
    for i, char in enumerate(curr_sentence):
        trainX[sent_idx, i, char_to_index[char]] = 1
    trainY[sent_idx, char_to_index[curr_next_char]] = 1

In [100]:
print(trainX.shape)
print(trainY.shape)

(48995, 40, 71)
(48995, 71)


In [117]:
model = Sequential()
model.add(LSTM(128, input_shape=(s_len, len(chars))))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

#train the model

#print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop')
model.fit(trainX, trainY, batch_size = 128, nb_epoch = 100)

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/100
48995/48995 [==============================] - 31s 625us/step - loss: 2.9846
Epoch 2/100
48995/48995 [==============================] - 29s 587us/step - loss: 2.4464
Epoch 3/100
48995/48995 [==============================] - 29s 595us/step - loss: 2.2371
Epoch 4/100
48995/48995 [==============================] - 30s 608us/step - loss: 2.1288
Epoch 5/100
48995/48995 [==============================] - 30s 603us/step - loss: 2.0560
Epoch 6/100
48995/48995 [==============================] - 28s 578us/step - loss: 2.0028
Epoch 7/100
48995/48995 [==============================] - 29s 590us/step - loss: 1.9546
Epoch 8/100
48995/48995 [==============================] - 28s 572us/step - loss: 1.9126
Epoch 9/100
48995/48995 [==============================] - 29s 595us/step - loss: 1.8793
Epoch 10/100
48995/48995 [==============================] - 27s 559us/step - loss: 1.8504 - ETA: 0s - loss:
Epoch 11/100
48995/48995 [==============================] - 28s 578us/step - loss: 1.8214
E

48995/48995 [==============================] - 28s 564us/step - loss: 0.8449
Epoch 91/100
48995/48995 [==============================] - 27s 551us/step - loss: 0.8386
Epoch 92/100
48995/48995 [==============================] - 27s 557us/step - loss: 0.8352
Epoch 93/100
48995/48995 [==============================] - 28s 574us/step - loss: 0.8217
Epoch 94/100
48995/48995 [==============================] - 29s 583us/step - loss: 0.8271
Epoch 95/100
48995/48995 [==============================] - 28s 570us/step - loss: 0.8180
Epoch 96/100
48995/48995 [==============================] - 28s 561us/step - loss: 0.8198
Epoch 97/100
48995/48995 [==============================] - 27s 548us/step - loss: 0.8149
Epoch 98/100
48995/48995 [==============================] - 29s 588us/step - loss: 0.8088
Epoch 99/100
48995/48995 [==============================] - 28s 569us/step - loss: 0.8084
Epoch 100/100
48995/48995 [==============================] - 29s 590us/step - loss: 0.8061


In [127]:
start_index = np.random.randint(0, len(text)-s_len-1)
start_index = 50
sentence = text[start_index: start_index + s_len]
sentence = "shall i compare thee to a summer's day?\n"
sequence = sentence
for i in range(1000):
    x_pred = np.zeros((1, s_len, len(chars)))
    for j, char in enumerate(sentence):
        x_pred[0, j, char_to_index[char]] = 1.
        
    predictions = np.array(model.predict(x_pred)[0])
    max_index = np.argmax(predictions)
    next_char = index_to_char[max_index]
    sentence = sentence[1:] + next_char
    sequence += next_char

print(sequence)

shall i compare thee to a summer's day?
Than should his bearth of love and thing disheded.
That heart of lect born with thy shall stear,
And to be cores me an me, and buty,
And such than should have fair of the love,
That this for to eare that with thy reserv,
And to nour livess wham I to be others,
And my to be buther bust which I doobs,
  And surmer best for my self sime, in sean,
And it himpith and did comprits of all
otherst,
And thin me branget me faill be than she dothee,
  Thou art his black and say the sum for I shade,
The eremy with belund is in the fire,
The base the reston shall batien to give his drade.
The hath in my sourngions on thy gract,
And to have to me besing and is thy sied,
The sean and thie still that thou shave, the owe,
  Thes rumbers no mann, will thou mint ewe,
  if to the surmpresse than she will grace,
Withingt thou thy sweet fare's now thou art,
Where in the shall wat my most mind eyes hear,
Whon thy shall be that foul such mise shell,
Whounst my self all 

In [111]:
index = np.random.choice(len(chars), 1, predictions.all())
print(index[0])

41


In [122]:
start_index = np.random.randint(0, len(text)-s_len-1)
start_index = 50
sentence = text[start_index: start_index + s_len]
sequence = sentence
for i in range(1000):
    x_pred = np.zeros((1, s_len, len(chars)))
    for j, char in enumerate(sentence):
        x_pred[0, j, char_to_index[char]] = 1.
        
    predictions = np.array(model.predict(x_pred)[0])
    #max_index = np.argmax(predictions)
    index = np.random.choice(len(chars), 1, p=predictions)[0]
    next_char = index_to_char[index]
    sentence = sentence[1:] + next_char
    sequence += next_char

print(sequence)

ire increase,
That thereby beauty's rose see flow Ind sweart
That of you sway woth from wey shy bideds,
And mire hele reain be rained in you.


                    12ed I camputain inding me ,
Teme in the weery wertting selfour sight
robt comber yot curnton to toke you sheart,
Whorlf at atbeasts and for mind ewe dig:ue,
atissred out doth ver,
at enit monks the rooner to the owers make,
Whoreming thee 'it stmer, frulless in thy hings,
And be2ty creets shame arver in the is detheed.
Thou a not thich my worch you bid, bul dead.
Then the eanth thou dongubhicg ofor mode,
And hate sungers of thy night,
How me, no or the vine hempastis wakl mus,
With uple miny of bain ampelsiof how rasth.
That pursh the thou my lies, why fordity nombull,
And loat of lais and not for shind soo
,
So waln I ar form ar monking heave.
When I an umenor bely faily done, Inong.
vend ny llone, without mondon merlinges ampare ince sweirnge,
When I ar mventer, and then me nom thou art.
  Ther that in my singll to mise m

In [56]:
print(len(x_pred))
print(len(predictions))
print(max(predictions))
print(np.argmax(predictions))
print(predictions)

1
71
0.730325
1
[  6.94062852e-04   7.30325460e-01   1.18445519e-06   7.78295682e-04
   1.11060372e-05   7.64163997e-06   2.44515250e-03   5.60624612e-05
   2.19699068e-04   5.26500116e-06   2.41895978e-05   3.95123561e-06
   5.21234870e-06   6.64439176e-06   5.94937501e-06   1.87032833e-06
   3.34175570e-06   3.50042365e-06   3.44382511e-06   8.43042362e-05
   1.56733108e-06   3.46993038e-05   2.34842213e-04   9.26001521e-05
   1.01433932e-06   5.02007242e-06   4.80299718e-07   6.68773864e-05
   6.11881489e-07   2.46848376e-05   1.70587271e-03   2.24313212e-09
   1.10972440e-08   1.80436655e-05   3.18466082e-05   2.87337753e-05
   6.46860281e-05   4.98809823e-06   9.20415971e-07   8.35577594e-05
   7.51566084e-04   7.88684019e-07   1.68324221e-09   1.43170066e-04
   4.98383315e-06   1.99320279e-02   3.33607686e-03   2.89679645e-03
   1.38006210e-02   2.89650504e-02   6.97758840e-03   1.96235511e-03
   1.81529578e-02   1.79779790e-02   2.02268620e-05   7.90904975e-04
   1.11462157e-02 